In [ ]:
import numpy as np
import random

# Configurazione
FUNCTIONS = ['+', '-', '*', '/', '**']
UNARY_FUNCTIONS = ['sin', 'cos', 'exp', 'log']
VARIABLES = ['x[0]', 'x[1]']
CONSTANTS = ['0', '1', '0.5', '2']

class Node:
    def __init__(self, value, children=None):
        """
        Inizializza un nodo dell'albero con un valore e figli opzionali.
        Converte i valori numerici in stringhe e valida il nodo.
        """
        # Conversione a stringa per valori numerici
        if isinstance(value, (int, float)):
            value = str(value)
        
        # Validazione del valore del nodo
        if (value not in VARIABLES and 
            value not in CONSTANTS and 
            value not in FUNCTIONS and   
            value not in UNARY_FUNCTIONS):
            raise ValueError(f"Valore del nodo non valido: {value}")
        
        self.value = value
        self.children = children or []
 
    def evaluate(self, x):
        """
        Valuta il nodo ricorsivamente, gestendo variabili, costanti, 
        funzioni binarie e funzioni unarie.
        """
        if self.value in VARIABLES:
            return eval(self.value)
        
        if self.value in CONSTANTS:
            return float(self.value)
        
        if self.value in FUNCTIONS:
            left_val = self.children[0].evaluate(x)
            right_val = self.children[1].evaluate(x)
            
            # Gestione della divisione per zero
            if self.value == '/' and right_val == 0:
                return 1.0  # Valore predefinito per divisione per zero
            
            # Valutazione dell'operazione binaria
            try:
                result = eval(f"{left_val} {self.value} {right_val}")
                # Controllo per evitare inf o NaN
                if not np.isfinite(result):
                    return 1.0
                return result
            except ZeroDivisionError:
                return 1.0
        
        if self.value in UNARY_FUNCTIONS:
            operand = self.children[0].evaluate(x)
            try:
                result = eval(f"np.{self.value}({operand})")
                if not np.isfinite(result):
                    return 1.0
                return result
            except ValueError:
                return 1.0
        
        raise ValueError(f"Valore del nodo sconosciuto: {self.value}")


    def to_numpy(self):
        """
        Converte l'albero in una stringa compatibile con NumPy.
        """
        if self.value in VARIABLES:
            return self.value
        
        if self.value in CONSTANTS:
            return str(self.value)
        
        if self.value in FUNCTIONS:
            return f"({self.children[0].to_numpy()} {self.value} {self.children[1].to_numpy()})"
        
        if self.value in UNARY_FUNCTIONS:
            return f"np.{self.value}({self.children[0].to_numpy()})"
        
        raise ValueError(f"Valore del nodo non convertibile: {self.value}")

def tree_to_list(tree):
    """
    Converte ricorsivamente un albero in una lista di nodi.
    """
    nodes = [tree]
    for child in tree.children:
        nodes.extend(tree_to_list(child))
    return nodes

def generate_random_tree(depth=0, max_depth=3):
    """
    Genera un albero sintattico casuale con profondità controllata.
    """
    # Condizione di arresto: raggiunta profondità massima o nodo foglia
    if depth >= max_depth or (depth > 0 and np.random.rand() < 0.3):
        value = np.random.choice(VARIABLES + CONSTANTS)
        return Node(value)
    
    # Generazione di nodo binario o unario
    if np.random.rand() < 0.7:  # Operatore binario
        operator = np.random.choice(FUNCTIONS)
        left = generate_random_tree(depth + 1, max_depth)
        right = generate_random_tree(depth + 1, max_depth)
        return Node(operator, [left, right])
    else:  # Funzione unaria
        unary_function = np.random.choice(UNARY_FUNCTIONS)
        child = generate_random_tree(depth + 1, max_depth)
        return Node(unary_function, [child])

def fitness(tree, x, y):
    """
    Calcola il fitness (errore quadratico medio) di un albero.
    """
    predictions = np.array([tree.evaluate(xi) for xi in x.T])
    return np.mean((predictions - y) ** 2)

def crossover(parent1, parent2):
    """
    Esegue il crossover tra due alberi genitori scambiando sottoalberi.
    """
    def copy_tree(node):
        if not node.children:
            return Node(node.value)
        return Node(node.value, [copy_tree(child) for child in node.children])
    
    child1, child2 = copy_tree(parent1), copy_tree(parent2)
    
    nodes1 = tree_to_list(child1)
    nodes2 = tree_to_list(child2)
    
    if nodes1 and nodes2:
        subtree1 = random.choice(nodes1)
        subtree2 = random.choice(nodes2)
        
        # Scambio dei valori e dei figli
        subtree1.value, subtree2.value = subtree2.value, subtree1.value
        subtree1.children, subtree2.children = subtree2.children, subtree1.children
    
    return child1, child2

def mutate(tree, mutation_rate=0.1):
    """
    Applica mutazione con una determinata probabilità.
    """
    if random.random() < mutation_rate:
        return generate_random_tree()
    return tree

def genetic_programming(x, y, population_size=50, generations=100):
    """
    Algoritmo di programmazione genetica per la regressione simbolica.
    """
    # Inizializzazione popolazione
    population = [generate_random_tree() for _ in range(population_size)]
    
    for generation in range(generations):
        # Calcolo fitness
        fitness_scores = [fitness(tree, x, y) for tree in population]
        
        # Miglior individuo
        best_index = np.argmin(fitness_scores)
        best_tree = population[best_index]
        print(f"Generazione {generation}: Miglior fitness = {fitness_scores[best_index]:.6f}")
        
        # Selezione e riproduzione
        new_population = []
        for _ in range(population_size // 2):
            # Selezione dei genitori con probabilità inversamente proporzionale al fitness
            parent1, parent2 = random.choices(population, weights=[1 / (f + 1) for f in fitness_scores], k=2)
            
            # Crossover e mutazione
            child1, child2 = crossover(parent1, parent2)
            new_population.extend([mutate(child1), mutate(child2)])
        
        population = new_population
    
    return best_tree

def generate_data(test_size=10_000, train_size=1_000):
    """
    Genera dati di input e output utilizzando una funzione definita.
    """
    def true_f(x: np.ndarray) -> np.ndarray:
        return x[0] + np.sin(x[1]) / 5

    # Generazione dati di validazione
    x_validation = np.vstack([
        np.random.random_sample(size=test_size) * 2 * np.pi - np.pi,  # Range [-π, π]
        np.random.random_sample(size=test_size) * 2 - 1,  # Range [-1, 1]
    ])
    y_validation = true_f(x_validation)

    # Selezione dati di training
    train_indexes = np.random.choice(test_size, size=train_size, replace=False)
    x_train = x_validation[:, train_indexes]
    y_train = y_validation[train_indexes]

    # Salvataggio dati
    np.savez("problem_0.npz", x=x_train, y=y_train)
    print("Dati generati e salvati in 'problem_0.npz'")

# Esecuzione principale
if __name__ == "__main__":
    # Genera dati
    generate_data()
    
    # Carica dati
    problem = np.load("problem_0.npz")
    x_train = problem['x']
    y_train = problem['y']
    
    # Esegui programmazione genetica
    best_tree = genetic_programming(x_train, y_train)
    
    # Esporta formula
    formula = best_tree.to_numpy()
    with open("s323914.py", "w") as f:
        f.write("import numpy as np\n")
        f.write(f"def f(x: np.ndarray) -> np.ndarray:\n    return {formula}\n")

Dati generati e salvati in 'problem_0.npz'


<string>:1: RuntimeWarning: invalid value encountered in log
<string>:1: RuntimeWarning: divide by zero encountered in log


Generazione 0: Miglior fitness = 0.010711
Generazione 1: Miglior fitness = 0.010711
Generazione 2: Miglior fitness = 0.010711
Generazione 3: Miglior fitness = 0.010711
Generazione 4: Miglior fitness = 0.010711
Generazione 5: Miglior fitness = 0.010711


<string>:1: RuntimeWarning: overflow encountered in exp
C:\Users\silvi\AppData\Local\Temp\ipykernel_9068\3953092817.py:124: RuntimeWarning: overflow encountered in square
  return np.mean((predictions - y) ** 2)


Generazione 6: Miglior fitness = 0.010711
Generazione 7: Miglior fitness = 0.010711
Generazione 8: Miglior fitness = 0.010711
Generazione 9: Miglior fitness = 0.010711
Generazione 10: Miglior fitness = 0.010711
Generazione 11: Miglior fitness = 0.010711
Generazione 12: Miglior fitness = 0.010711
Generazione 13: Miglior fitness = 0.010711
Generazione 14: Miglior fitness = 0.010711
Generazione 15: Miglior fitness = 0.010711
Generazione 16: Miglior fitness = 0.010711
Generazione 17: Miglior fitness = 0.010711
Generazione 18: Miglior fitness = 0.010711


OverflowError: (34, 'Result too large')